In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))


/var/folders/lj/2t8twtsx5v1dw1dkbr59j1xr0000gn/T/ipykernel_65699/3522731578.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [8]:
# Divisional Round EV Calculator

import mysql.connector
from collections import defaultdict

# Database connection
conn = mysql.connector.connect(
    host='betting-db.cp86ssaw6cm7.us-east-1.rds.amazonaws.com',
    user='admin',
    password='7nRB1i2&A-K>',
    database='betting_db'
)

cursor = conn.cursor()

# Query to fetch Potential Payouts for Conference, Championship, and Quarterfinals
query = """
    SELECT 
        legs.ParticipantName,
        legs.EventType,
        SUM(bets.PotentialPayout) AS total_payout
    FROM bets
    JOIN legs ON bets.WagerID = legs.WagerID
    WHERE 
        bets.LegCount = 1
        AND bets.WLCA = 'Active'
        AND bets.WhichBankroll = 'GreenAleph'
        AND legs.LeagueName = 'NFL'
        AND legs.EventType IN ('Conference Winner', 'Championship', 'Quarterfinals')
    GROUP BY legs.ParticipantName, legs.EventType;
"""

cursor.execute(query)

# Populate payouts dictionary with normalization
payouts = defaultdict(lambda: {'payout_conference': 0, 'payout_championship': 0, 'payout_quarterfinals': 0})
for participant_name, event_type, total_payout in cursor.fetchall():
    normalized_name = participant_name.strip().lower()  # Normalize name
    if event_type.strip().lower() == "conference winner":
        normalized_event_type = "conference"  # Map explicitly to 'conference'
    else:
        normalized_event_type = event_type.strip().lower().replace(" ", "_")  # Normalize event type
    payouts[normalized_name][f'payout_{normalized_event_type}'] = float(total_payout)

cursor.close()
conn.close()

# Matchup structure with sample team names
matchups = {
    "AFC": [
        ("Baltimore Ravens", "Buffalo Bills"),
        ("Houston Texans", "Kansas City Chiefs")
    ],
    "NFC": [
        ("Los Angeles Rams", "Philadelphia Eagles"), 
        ("Washington Commanders", "Detroit Lions")
    ]
}

# Probabilities and payouts for each team (from FanDuel)
team_probabilities = {
    "buffalo bills": {'current_round_prob': 0.483, 'current_quarterfinals_prob': 0.40, 'current_conference_prob': 0.27, 'current_champ_prob': 0.14},
    "baltimore ravens": {'current_round_prob': 0.517, 'current_quarterfinals_prob': 0.41, 'current_conference_prob': 0.30, 'current_champ_prob': 0.155},
    "kansas city chiefs": {'current_round_prob': 0.807, 'current_quarterfinals_prob': 0.40, 'current_conference_prob': 0.40, 'current_champ_prob': 0.205},
    "houston texans": {'current_round_prob': 0.193, 'current_quarterfinals_prob': 0.15, 'current_conference_prob': 0.03, 'current_champ_prob': 0.01},
    "detroit lions": {'current_round_prob': 0.813, 'current_quarterfinals_prob': 0.34, 'current_conference_prob': 0.475, 'current_champ_prob': 0.2425},
    "washington commanders": {'current_round_prob': 0.187, 'current_quarterfinals_prob': 0.12, 'current_conference_prob': 0.065, 'current_champ_prob': 0.0225},
    "philadelphia eagles": {'current_round_prob': 0.727, 'current_quarterfinals_prob': 0.34, 'current_conference_prob': 0.35, 'current_champ_prob': 0.18},
    "los angeles rams": {'current_round_prob': 0.273, 'current_quarterfinals_prob': 0.22, 'current_conference_prob': 0.11, 'current_champ_prob': 0.045}
}

# Normalize team probabilities
team_probabilities = {k.strip().lower(): v for k, v in team_probabilities.items()}

# EV calculation functions
def calculate_ev(probability, payout):
    return probability * payout

def calculate_conditional_ev(current_quarterfinals_prob, quarterfinals_payout, current_conference_prob, conference_payout, current_champ_prob, champ_payout, current_round_prob):
    if current_round_prob > 0:
        conditional_quarterfinals_prob = current_quarterfinals_prob / current_round_prob
        conditional_conference_prob = current_conference_prob / current_round_prob
        conditional_champ_prob = current_champ_prob / current_round_prob
    else:
        return 0, 0, 0  # Avoid division by zero

    quarterfinals_ev = calculate_ev(conditional_quarterfinals_prob, quarterfinals_payout)
    conference_ev = calculate_ev(conditional_conference_prob, conference_payout)
    champ_ev = calculate_ev(conditional_champ_prob, champ_payout)

    return quarterfinals_ev, conference_ev, champ_ev

# Loop through each conference and matchup, calculating EVs
for conference, games in matchups.items():
    print(f"\n{conference} Conference Second-Round Matchups:\n")
    for team1_name, team2_name in games:
        team1_key = team1_name.strip().lower()
        team2_key = team2_name.strip().lower()

        team1 = {
            "current_round_prob": team_probabilities[team1_key]["current_round_prob"],
            "current_quarterfinals_prob": team_probabilities[team1_key]["current_quarterfinals_prob"],
            "current_conference_prob": team_probabilities[team1_key]["current_conference_prob"],
            "current_champ_prob": team_probabilities[team1_key]["current_champ_prob"],
            "quarterfinals_payout": payouts[team1_key]["payout_quarterfinals"],
            "conference_payout": payouts[team1_key]["payout_conference"],
            "champ_payout": payouts[team1_key]["payout_championship"]
        }

        team2 = {
            "current_round_prob": team_probabilities[team2_key]["current_round_prob"],
            "current_quarterfinals_prob": team_probabilities[team2_key]["current_quarterfinals_prob"],
            "current_conference_prob": team_probabilities[team2_key]["current_conference_prob"],
            "current_champ_prob": team_probabilities[team2_key]["current_champ_prob"],
            "quarterfinals_payout": payouts[team2_key]["payout_quarterfinals"],
            "conference_payout": payouts[team2_key]["payout_conference"],
            "champ_payout": payouts[team2_key]["payout_championship"]
        }

        # Calculate conditional EVs
        team1_qf_ev, team1_cf_ev, team1_champ_ev = calculate_conditional_ev(
            team1["current_quarterfinals_prob"], team1["quarterfinals_payout"],
            team1["current_conference_prob"], team1["conference_payout"],
            team1["current_champ_prob"], team1["champ_payout"],
            team1["current_round_prob"]
        )

        team2_qf_ev, team2_cf_ev, team2_champ_ev = calculate_conditional_ev(
            team2["current_quarterfinals_prob"], team2["quarterfinals_payout"],
            team2["current_conference_prob"], team2["conference_payout"],
            team2["current_champ_prob"], team2["champ_payout"],
            team2["current_round_prob"]
        )

        # Total EVs
        team1_ev_win = team1_qf_ev + team1_cf_ev + team1_champ_ev
        team2_ev_win = team2_qf_ev + team2_cf_ev + team2_champ_ev

        # Calculate Series Delta
        series_delta = abs(team1_ev_win - team2_ev_win)

        # Output results
        print(f"{team1_name} vs {team2_name}")
        print(f"{team1_name} EV if win round: ${team1_ev_win:.2f}")
        print(f"  Quarterfinals EV: ${team1_qf_ev:.2f}")
        print(f"  Conference EV: ${team1_cf_ev:.2f}")
        print(f"  Championship EV: ${team1_champ_ev:.2f}")
        print(f"{team2_name} EV if win round: ${team2_ev_win:.2f}")
        print(f"  Quarterfinals EV: ${team2_qf_ev:.2f}")
        print(f"  Conference EV: ${team2_cf_ev:.2f}")
        print(f"  Championship EV: ${team2_champ_ev:.2f}")
        print(f"Matchup Delta: ${series_delta:.2f}\n")



AFC Conference Second-Round Matchups:

Baltimore Ravens vs Buffalo Bills
Baltimore Ravens EV if win round: $19670.31
  Quarterfinals EV: $0.00
  Conference EV: $0.00
  Championship EV: $19670.31
Buffalo Bills EV if win round: $19514.49
  Quarterfinals EV: $0.00
  Conference EV: $0.00
  Championship EV: $19514.49
Matchup Delta: $155.82

Houston Texans vs Kansas City Chiefs
Houston Texans EV if win round: $10279.79
  Quarterfinals EV: $0.00
  Conference EV: $0.00
  Championship EV: $10279.79
Kansas City Chiefs EV if win round: $66380.36
  Quarterfinals EV: $0.00
  Conference EV: $9206.44
  Championship EV: $57173.92
Matchup Delta: $56100.57


NFC Conference Second-Round Matchups:

Los Angeles Rams vs Philadelphia Eagles
Los Angeles Rams EV if win round: $113600.73
  Quarterfinals EV: $0.00
  Conference EV: $3062.27
  Championship EV: $110538.46
Philadelphia Eagles EV if win round: $16388.17
  Quarterfinals EV: $0.00
  Conference EV: $0.00
  Championship EV: $16388.17
Matchup Delta: $972